In [84]:
import requests
import json
from tqdm import tqdm

import pandas as pd

In [ ]:
https://geoscience.data.qld.gov.au/api/action/package_search?q=seismic&rows=1000&start=0

In [ ]:
# set the API endpoint
api = 'https://geoscience.data.qld.gov.au/api/action/'
# construct our query
query = api + 'package_search?q=seismic&rows=1000'
# make the get request and store it in the response object
response = requests.get(query)
# view the payload as JSON
json_response = response.json()

#and get a count of results we can retrieve
total_results = json_response['result']['count']

#hack without understanding/researching the api too much
getloop = int(total_results/1000) + 1

results_list = []
for i in range(getloop):
    loopq = query + "&start=" + str(i*1000)
    response = requests.get(loopq)
    json_response = response.json()
    
    results_list.append(json_response)

# print the GeoJSON for the results
#for index, dataset in enumerate(json_response['result']['results']):
    #print(dataset.get('GeoJSONextent'))
    #print(index)

In [ ]:
#look at a result for the structure
results_list[0]['result']['results'][887]

In [ ]:
#each one has organisation - don't really need
#id, , author, spatial, type is useful
#resources - a list of dicts, 
#eg resource:description:
#resource description dictionary
#a resource id, resource:size, resource:, download_url, name, url, resource_type etc

In [ ]:
#will try and make a dataframe
#but using the batched results, getting the ids
#turning the resource dictionaries into dataframes, concatenating
#then concatenating back at the end

In [ ]:
result_ids = []
for batch in tqdm(results_list, total=len(results_list) ):
    results = batch['result']['results']
    for r in results:
        rid = r['id']
        rauthor = r['author']
        rspatial = r['spatial']
        rtype = r['type']
        
        if len(r['resources']) > 0:
            part_list = []
            for part in r['resources']:
                #df = pd.DataFrame.from_dict(part)
                df = pd.DataFrame([part])
                df['rid'] = rid
                df['rauthor'] = rauthor
                df['rspatial'] = rspatial
                df['type'] = rtype
                part_list.append(df)
                allparts = pd.concat(part_list)
                
            result_ids.append(allparts)

dfDatasets = pd.concat(result_ids)            

In [90]:
dfDatasets.to_csv(r'H:\GSQ_Seimisc_Datasets.csv')

In [63]:
#this is what a default query would look like
query_default = api + 'package_search?q=seismic

response = requests.get(query_default)
json_response = response.json()

print(json_response['result']['count'])
print(len(json_response['result']['results']) )
for x in json_response['result']['results']:
    print(x["id"])

10
4267
aaa7c8c3-d69b-4ae6-bc4b-2542301b7e6e
39bc2c60-d727-4383-b82d-6af7fdf8cf7b
2f3d761c-6506-438a-934a-87e48cc8d25e
8a1f58c4-2e61-44a5-a573-794396652ea6
55a1c3a1-107f-4a65-9bec-bef9e6eb8d6b
8157408f-83af-4515-9279-f502ae16c155
37a2fb0c-49c5-444a-a7da-4947ae9dad94
3897db72-0d38-4f11-aad4-8fa8907d20bb
4d5ae331-3cfb-4042-a4b5-1955c135cace
30b6b5fc-ba25-4bfe-a76b-c35d72ba07e0


Check how many are of type seismic, not report

In [95]:
dfDatasets.loc[dfDatasets['type'] == 'seismic']['rid'].unique().size

8